# VarArgs, KwArgs, значения аргументов по умолчанию

Возможность передавать произвольное количество аргументов и именованные аргументы

In [4]:
def fd(a, b, c=3):
    print(a, b, c)

fv(1, 2, 3)

def fv(a, b, c, *v, **k):
    print(a, b, c, v, k)

fv(1, 2, 3, 4, 5, x=1, y=2, z=3)
fv(1, 2, 3, *[-10, -20, -30], **{'Э': 45, 'Ю': 67})

1 2 3 () {}
1 2 3 (4, 5) {'x': 1, 'y': 2, 'z': 3}
1 2 3 (-10, -20, -30) {'Э': 45, 'Ю': 67}


И показывать фокусы

In [9]:
matrix = [[11, 12, 13],
          [21, 22, 23],
          [31, 32, 33]
         ]

print(zip([1, 2, 3], ['a', 'b', 'c']))

transposed = zip(*matrix)

print(list(transposed))

[(11, 21, 31), (12, 22, 32), (13, 23, 33)]


# Декораторы

Людей много, и они все разные. Как бы это ни звучало, но общество требует (обычно от государства) с такими разными людьми делать, тем не менее, одинаковые стандартные вещи. Например, регистрировать людей в фискальных органах, платить пенсионерам пенсию, делать детям прививки от кори. За отдельными исключениями, это стандартное действие.

Функции тоже все разные. Стандартная операция — вызвать функцию. Ещё её можно использовать как значение, чтобы куда-нибудь передать. Удобная операция (уже требующая использовать функцию, как значение) — декорирование. Перед тем, как получить своё имя, функцию «пропускают» через другую функцию.

Например, мы хотим, чтобы функция перед вызовом печатала свой параметр (для простоты будем считать, что он один), а после выхода — возвращаемое значение. Так тому и быть.

## Пример

In [1]:
def test_function(n):
    return n * 2

def logged(genuine_function):  # Эта функция реализует декоратор. Она получает на вход исходную (настоящую) функцию
    def fake_function(argument):  # Создаёт внутри фальшивую функцию
        result = genuine_function(argument) # Считает значение настоящей функции
        print("Функция:", repr(genuine_function), ", аргумент:", argument, ", значение:", result)  # Печатает информацию
        return result  # И возвращает значение
    return fake_function  # Ещё не забыли? Декоратор получает функцию и возвращает тоже функцию

test_function = logged(test_function)  # Пропускаем test_function через декоратор и запоминаем под тем же именем

print(test_function(5))

Функция: <function test_function at 0x000001463F4D1400> , аргумент: 5 , значение: 10
10


Можно и поизящнее — для этого есть специальный удобный синтаксис:

In [2]:
@logged
def test_function_2(n):
    return n * 3

print(test_function(3))

Функция: <function test_function at 0x000001463F4D1400> , аргумент: 3 , значение: 6
6


## А зачем вообще всё это нужно?

* Декораторы `@staticmethod` и `@classmethod`, которые позволяют вызывать статический метод от экземпляра объекта (удобно!). Но про классы и объекты позже.
* Регистрация функций, как обработчиков событий, например, в веб-приложениях (пример с фреймфорком Flask):

```
rom flask import Flask
app = Flask(__name__)

@app.route('/hello')
def hello_world():
    return 'Hello, World!'
```

После этого по запросу `http://сервер/hello` браузер получит обратно строку "Hello, World!". Т.е. декоратор саму функцию не «переделывает», зато сообщает фреймворку, что она понадобится в такой-то ситуации.

* Т.н. *мемоизация* — кеширование результатов функций **без побочных эффектов**, чтобы не считать многократно то, что уже посчитано, но нужно часто. Подробнее см. [`@functools.lru_cache`](https://docs.python.org/3/library/functools.html#functools.lru_cache).

* Изменение свойств функций, как в нашем примере

Стоп. А мы реально хотим, чтобы прямо на консоль (точнее в `STDOUT`) выдавался весь наш лог — кого с какими параметрами вызвали?.. Так себе решение. Давайте сделаем декоратор, который позволит писать в произвольный поток. В нашем примере будем писать в `STDERR`, но это легко переделать под запись в файл на диске, например. Приступим.

In [3]:
import sys

def stream_logged(stream: 'file'):
    """Log function arguments and value to stream"""
    # Эта функция — не декоратор. Получив аргумент, она создаст и вернёт декоратор!

    # А дальше — бессовестный копипаст сверху
    def logging_decorator(genuine_function):  # Эта функция реализует декоратор. Она получает на вход исходную (настоящую) функцию
        def fake_function(argument):  # Создаёт внутри фальшивую функцию
            result = genuine_function(argument) # Считает значение настоящей функции
            print("Функция:", repr(genuine_function), ", аргумент:", argument, ", значение:", result, file=stream)  # Печатает информацию
            return result  # И возвращает значение
        return fake_function  # Ещё не забыли? Декоратор получает функцию и возвращает тоже функцию

    return logging_decorator # Ну и наконец выдайм сам декоратор

# Чтобы получить декоратор, мы вызываем stream_logged от параметра —
# файлового объекта. Полученный декоратор уже применяется к функции.
# А для «потребителя» выглядит просто как декоратор с параметром...
@stream_logged(sys.stderr) 
def rev(s):
    return list(s) == list(reversed(s))

print(rev("арозаупаланалапуазора"))

True


Функция: <function rev at 0x000001463F4D1D08> , аргумент: арозаупаланалапуазора , значение: True


Напоминаетм: то, что оно красненькое — это не ошибка. Это последствие нашего желания печатать в `STDERR`.

## Задание на декораторы

Ну и как без него?

Сделать «декоратор с параметром» (но мы-то знаем), который будет применять функцию к арументу заданное количество раз. Т.е. такой:

```
def repeat(n):  # Вот это и надо реализовать
    ...
    ...


@repeat(2)
def plus_1(x):
    return x + 1


@repeat(0)
def mul_2(x):
    return x * 2

print(plus_1(3))  # должно выдать 5
print(mul_2(4))  # должно выдать 4
```

# Статическая типизация и typechecker'ы

У всех уже возникала хоть одна ошибка, связанная с неожиданными типами значений переменных? Скорее всего да, т.к. Питон — язык динамически типизированный, и контроля типов не проводит, так что ошибки становятся видны «в последний момент».

Для коротких программ это обычно не критчно, для длинных — имеет смысл подойти к вопросу основательно. Встречаем `typing`!

In [4]:
# Импортируем некотрые имена из модуля typing
from typing import List, Union, Tuple, Dict

# Просто аннотации. Изначально их создавали для документирвоания кода
def function_0(i: "Привет") -> "Пока":
    return -i


# Но довольно быстро обнаружилось, что их синтаксис слишком уж напоминает типы переменных и параметров в других языках
# И их стали писать для того, чтобы указывать на типы
def function_1(i: int, j: int) -> int:
    return i // j

print(function_1(1, 2))

# Типы можно конструировать из других типов (ещё бы!) Статически типизированные языки это позволяют и даже требуют.
# Пример посложнее:
def function_2(d: Dict[Tuple[str, int], List[Union[int, str]]]) -> None:
    d["abc", 123] = [1, "2", 3, "4"]

d: Dict[Tuple[str, int], List[Union[int, str]]] = dict()
function_2(d)
print(d)

# А можно страшный тип как-то назвать, и потом просто писать t вместо этой длинной жути
t = Dict[Tuple[str, int], List[Union[int, str]]]

def function_3(d: t) -> None:
    d["zzz", 456] = []

function_3(d)
print(d)

0
{('abc', 123): [1, '2', 3, '4']}
{('abc', 123): [1, '2', 3, '4'], ('zzz', 456): []}


Вроде всё работает, если не поломать? А зачем?

Давайте программку сохраним в файл `type_check.py`

И выполним:

```
pip install mypy
mypy type_check.py
```

MyPy — статический верификатор типов для Python. Помогает.

# PyLint

А ещё есть такая штука — PyLint.

```
pip install pylint
pylint type_check.py
```